# 01. 라이브러리 임포트

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from math import sqrt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# plt.rcParams['font.family'] = 'Malgun Gothic'    # 윈도우
plt.rcParams['font.family'] = 'AppleGothic'        # 맥
# plt.rcParams['font.family'] = 'NanumGothic'      # 리눅스
plt.rcParams['axes.unicode_minus'] = False         # 마이너스(-) 깨짐 현상 방지

# 01. 데이터 로드

In [2]:
df = pd.read_csv("../data/processed/survey_2024_winsor.csv")

print(f"데이터 형태: {df.shape}")
print(f"컬럼 수: {len(df.columns)}")

데이터 형태: (15797, 116)
컬럼 수: 116


# 02. 웰니스와 비웰니스 구분

In [3]:
# 분석 대상 컬럼 정의
wellness_col = "Q8a04"  # 휴양/휴식(웰니스) 활동 참여 여부
group_0 = df[df[wellness_col] == 0]  # 휴양/휴식(웰니스) 참여 안함
group_1 = df[df[wellness_col] == 1]  # 휴양/휴식(웰니스) 참여함

print("=== 휴양/휴식(웰니스) 활동 참여 집단 분석 ===\n")
print(f"전체 응답자: {len(df):,}명")
print(f"집단 0 (참여 안함): {len(group_0):,}명 ({len(group_0)/len(df)*100:.1f}%)")
print(f"집단 1 (참여함): {len(group_1):,}명 ({len(group_1)/len(df)*100:.1f}%)\n")

=== 휴양/휴식(웰니스) 활동 참여 집단 분석 ===

전체 응답자: 15,797명
집단 0 (참여 안함): 13,206명 (83.6%)
집단 1 (참여함): 2,591명 (16.4%)



# 03. T-test (연속형 변수) / 로지스틱 회귀

## 총 체재기간_61일 이상 결측

In [4]:
from scipy import stats

target_col = "M일HAP_61"  # 총 체재기간

# 기술통계량
print("=== 체재기간 기술통계 ===")
print("참여 안함:", group_0[target_col].describe())
print("참여 함:", group_1[target_col].describe(), "\n")

# 등분산 검정 (Levene's test)
levene_stat, levene_p = stats.levene(group_0[target_col].dropna(),
                                     group_1[target_col].dropna())
print("Levene 등분산 검정")
print(f"통계량={levene_stat:.3f}, p-value={levene_p:.3f}\n")

# t-test
t_stat, p_value = stats.ttest_ind(group_0[target_col].dropna(),
                                  group_1[target_col].dropna(),
                                  equal_var=(levene_p > 0.05))  # 등분산 가정 여부 반영
print("t-test 결과")
print(f"t-statistic={t_stat:.3f}, p-value={p_value:.3f}")

if p_value < 0.05:
    print("=> 두 집단 간 체재기간 평균 차이는 통계적으로 유의함 (p<0.05)")
else:
    print("=> 두 집단 간 체재기간 평균 차이는 통계적으로 유의하지 않음")

=== 체재기간 기술통계 ===
참여 안함: count    13206.000000
mean         7.201575
std          5.754997
min          1.000000
25%          4.000000
50%          5.000000
75%          8.000000
max         35.000000
Name: M일HAP_61, dtype: float64
참여 함: count    2591.000000
mean        9.155924
std         6.955984
min         1.000000
25%         5.000000
50%         7.000000
75%        11.000000
max        35.000000
Name: M일HAP_61, dtype: float64 

Levene 등분산 검정
통계량=96.051, p-value=0.000

t-test 결과
t-statistic=-13.428, p-value=0.000
=> 두 집단 간 체재기간 평균 차이는 통계적으로 유의함 (p<0.05)


In [5]:
y = df[wellness_col]
X = df[["M일HAP_61"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.439516
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15793
Method:                           MLE   Df Model:                            3
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                 0.01514
Time:                        16:48:33   Log-Likelihood:                -6943.0
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 5.088e-46
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0839      0.075    -27.802      0.000      -2.231      -1.937
M일HAP_61       0.0446      0.

로지스틱 회귀분석 결과, 체재기간이 길수록 웰니스 활동 참여 확률이 유의하게 증가하였다(OR=1.05, p<0.001).

또한 여성의 참여 확률이 남성보다 높았으나, 연령은 유의한 영향을 보이지 않았다.

이는 웰니스 관광이 장기 체류 성향과 밀접히 연결되어 있음을 시사한다.

## 1인 지출경비(상하위 1% 대체)

In [6]:
target_col = "총액1인TOT2"  # 1인 지출경비

# 기술통계량
print("=== 1인 지출경비 기술통계 ===")
print("참여 안함:", group_0[target_col].describe())
print("참여 함:", group_1[target_col].describe(), "\n")

# 등분산 검정 (Levene's test)
levene_stat, levene_p = stats.levene(group_0[target_col].dropna(),
                                     group_1[target_col].dropna())
print("Levene 등분산 검정")
print(f"통계량={levene_stat:.3f}, p-value={levene_p:.3f}\n")

# t-test
t_stat, p_value = stats.ttest_ind(group_0[target_col].dropna(),
                                  group_1[target_col].dropna(),
                                  equal_var=(levene_p > 0.05))  # 등분산 가정 여부 반영
print("독립표본 t-test 결과")
print(f"t-statistic={t_stat:.3f}, p-value={p_value:.3f}")

if p_value < 0.05:
    print("=> 두 집단 간 1인 지출경비 평균 차이는 통계적으로 유의함 (p<0.05)")
else:
    print("=> 두 집단 간 1인 지출경비 평균 차이는 통계적으로 유의하지 않음")

=== 1인 지출경비 기술통계 ===
참여 안함: count    13206.000000
mean      1995.944320
std       1566.562393
min        335.520000
25%        983.500000
50%       1515.683333
75%       2470.832500
max       9413.940000
Name: 총액1인TOT2, dtype: float64
참여 함: count    2591.000000
mean     2199.791167
std      1544.720772
min       335.520000
25%      1211.089500
50%      1792.000000
75%      2700.000000
max      9413.940000
Name: 총액1인TOT2, dtype: float64 

Levene 등분산 검정
통계량=0.022, p-value=0.883

독립표본 t-test 결과
t-statistic=-6.070, p-value=0.000
=> 두 집단 간 1인 지출경비 평균 차이는 통계적으로 유의함 (p<0.05)


In [7]:
y = df[wellness_col]
X = df[["총액1인TOT2"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.444630
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15793
Method:                           MLE   Df Model:                            3
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                0.003683
Time:                        16:48:33   Log-Likelihood:                -7023.8
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 3.099e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8052      0.073    -24.888      0.000      -1.947      -1.663
총액1인TOT2    7.953e-05   1.27e

로지스틱 회귀분석 결과, 1인 지출경비가 증가할수록 웰니스 활동 참여 확률이 유의하게 높아졌다(OR≈2.2, 10만 원 증가 기준, p<0.001).

또한 여성의 참여 확률이 남성보다 높게 나타났으나, 연령은 유의한 영향을 보이지 않았다.

이는 웰니스 관광이 소비 성향이 큰 관광객과 여성 집단에서 특히 활발히 이루어짐을 보여준다.

## 1인 1일 지출경비(상하위 1% 대체, 61일 이상 결측)

In [8]:
target_col = "MDAY전체TOT_RAW61"  # 1인 지출경비

# 기술통계량
print("=== 1인 1일 지출경비 기술통계 ===")
print("참여 안함:", group_0[target_col].describe())
print("참여 함:", group_1[target_col].describe(), "\n")

# 등분산 검정 (Levene's test)
levene_stat, levene_p = stats.levene(group_0[target_col].dropna(),
                                     group_1[target_col].dropna())
print("Levene 등분산 검정")
print(f"통계량={levene_stat:.3f}, p-value={levene_p:.3f}\n")

# t-test
t_stat, p_value = stats.ttest_ind(group_0[target_col].dropna(),
                                  group_1[target_col].dropna(),
                                  equal_var=(levene_p > 0.05))  # 등분산 가정 여부 반영
print("독립표본 t-test 결과")
print(f"t-statistic={t_stat:.3f}, p-value={p_value:.3f}")

if p_value < 0.05:
    print("=> 두 집단 간 1인 1일 지출경비 평균 차이는 통계적으로 유의함 (p<0.05)")
else:
    print("=> 두 집단 간 1인 1일 지출경비 평균 차이는 통계적으로 유의하지 않음")

=== 1인 1일 지출경비 기술통계 ===
참여 안함: count    13206.000000
mean       333.877508
std        268.008502
min         22.000000
25%        183.985500
50%        260.039000
75%        383.333333
max       1687.500000
Name: MDAY전체TOT_RAW61, dtype: float64
참여 함: count    2591.000000
mean      304.465847
std       243.115903
min        22.000000
25%       168.211024
50%       251.740000
75%       362.583333
max      1687.500000
Name: MDAY전체TOT_RAW61, dtype: float64 

Levene 등분산 검정
통계량=7.454, p-value=0.006

독립표본 t-test 결과
t-statistic=5.534, p-value=0.000
=> 두 집단 간 1인 1일 지출경비 평균 차이는 통계적으로 유의함 (p<0.05)


In [9]:
y = df[wellness_col]
X = df[["MDAY전체TOT_RAW61"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.444983
         Iterations 6


                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15793
Method:                           MLE   Df Model:                            3
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                0.002893
Time:                        16:48:33   Log-Likelihood:                -7029.4
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 7.227e-09
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -1.5318      0.071    -21.494      0.000      -1.671      -1.392
MDAY전체TOT_RAW61    -0.0005   9.26e-05     -4.914      0.000      -0.001      -0.000
D_SEX               0.1461      

로지스틱 회귀분석 결과, 1인 1일 지출경비가 증가할수록 웰니스 활동 참여 확률은 유의하게 감소하였다(OR=0.9995, p<0.001).

또한 여성의 참여 확률이 남성보다 높게 나타났으나, 연령은 유의한 영향을 보이지 않았다.

흥미롭게도, 총액 지출은 참여 확률을 높이는 요인으로 나타난 반면, 1일 지출은 참여 확률을 낮추는 결과가 확인되었다.

이는 웰니스 관광이 고비용 소비 집단보다는 체류기간을 늘리면서 합리적으로 지출하는 관광객에게 더 적합한 활동임을 보여준다.

# 04. 카이제곱검정 / 로지스틱 회귀

## 성별

In [10]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)
var = "D_SEX"            # 방한횟수

# 교차표
ct = pd.crosstab(df[wellness_col], df[var])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 성별 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 성별 ===
D_SEX     0     1
Q8a04            
0      5599  7607
1      1000  1591 

Chi2=12.718, dof=1, p-value=0.0003621


In [11]:
wellness_col = "Q8a04"

y = df[wellness_col]
X = df[["D_SEX"]].copy()

X = sm.add_constant(X)  # 상수항 추가

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.445864
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15795
Method:                           MLE   Df Model:                            1
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:               0.0009190
Time:                        16:48:34   Log-Likelihood:                -7043.3
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 0.0003187
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7226      0.034    -50.176      0.000      -1.790      -1.655
D_SEX          0.1579      0.

단순 로지스틱 회귀분석에서 여성 관광객은 남성보다 웰니스 활동 참여 확률이 유의하게 높았다(OR=1.17, p<0.001).

그러나 성별 변수 단독으로는 설명력이 매우 낮아, 웰니스 참여는 성별 외에도 다양한 요인에 의해 영향을 받음을 알 수 있다.

## 연령

- 1 : 15~19세
- 2 : 20대
- 3 : 30대
- 4 : 40대
- 5 : 50대
- 6 : 60대 이상

In [12]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)
var = "D_AGE"            # 연령

# 교차표
ct = pd.crosstab(df[wellness_col], df[var])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 연령 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 연령 ===
D_AGE    1     2     3     4     5    6
Q8a04                                  
0      279  4125  3471  2454  2267  610
1       51   870   673   433   451  113 

Chi2=8.587, dof=5, p-value=0.1267


In [13]:
wellness_col = "Q8a04"

y = df[wellness_col]
X = df[["D_AGE"]].copy()

X = sm.add_constant(X)  # 상수항 추가

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.446197
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15795
Method:                           MLE   Df Model:                            1
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:               0.0001725
Time:                        16:48:34   Log-Likelihood:                -7048.6
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                    0.1188
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5408      0.060    -25.629      0.000      -1.659      -1.423
D_AGE         -0.0267      0.

단순 로지스틱 회귀분석에서 연령이 높을수록 웰니스 활동 참여 확률이 낮아지는 경향을 보였으나(OR=0.97), 이는 통계적으로 유의하지 않았다(p=0.119).

따라서 연령 단독 변수만으로 웰니스 참여 여부를 설명하기는 어렵다.

## 여행 형태

In [14]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)

# 여행형태 범주 생성
df["TYP_CAT"] = df[["TYP1", "TYP2", "TYP3"]].idxmax(axis=1)

# 교차표
ct = pd.crosstab(df[wellness_col], df["TYP_CAT"])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 여행형태 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 여행형태 ===
TYP_CAT   TYP1  TYP2  TYP3
Q8a04                     
0        10935   728  1543
1         2246    34   311 

Chi2=83.325, dof=2, p-value=8.059e-19


In [15]:
y = df[wellness_col]

# 여행형태
X = df[["TYP2", "TYP3"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.442150
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15792
Method:                           MLE   Df Model:                            4
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                0.009240
Time:                        16:48:34   Log-Likelihood:                -6984.6
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 3.389e-27
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6256      0.068    -23.940      0.000      -1.759      -1.493
TYP2          -1.5155      0.

로지스틱 회귀분석 결과, 여행형태와 성별이 웰니스 참여에 유의한 영향을 미쳤다.

개별여행(TYP1)을 선택한 관광객은 AIR-TEL 여행(TYP2) 관광객보다 웰니스 활동에 참여할 가능성이 현저히 높았으며(OR=0.22, p<0.001),

여성은 남성보다 웰니스 활동 참여 확률이 약 20% 더 높았다(OR=1.21, p<0.001).

반면 단체여행(TYP3)은 개별여행(TYP1)과 차이가 없었고, 연령 역시 통계적으로 유의하지 않았다.

## 방한목적별

In [16]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)

# 방한목적 범주 생성
df["TYP_CAT"] = df[["D_MOK1", "D_MOK2", "D_MOK3", "D_MOK4", "D_MOK5"]].idxmax(axis=1)

# 교차표
ct = pd.crosstab(df[wellness_col], df["TYP_CAT"])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 방한목적 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 방한목적 ===
TYP_CAT  D_MOK1  D_MOK2  D_MOK3  D_MOK4  D_MOK5
Q8a04                                          
0          8381    1437    2798     414     176
1          1775     533     180      69      34 

Chi2=406.227, dof=4, p-value=1.255e-86


In [17]:
y = df[wellness_col]

# 방한목적
X = df[["D_MOK2", "D_MOK3", "D_MOK4", "D_MOK5"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.431999
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15790
Method:                           MLE   Df Model:                            6
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                 0.03199
Time:                        16:48:34   Log-Likelihood:                -6824.3
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 3.012e-94
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4550      0.069    -21.198      0.000      -1.590      -1.320
D_MOK2         0.5614      0.

로지스틱 회귀분석 결과, 방한 목적이 웰니스 관광 참여에 중요한 영향을 미쳤다.

친구·친지 방문(D_MOK2) 관광객은 여가·위락 목적(D_MOK1) 관광객보다 웰니스 활동 참여 확률이 1.75배 높았던 반면,

사업·전문활동 목적(D_MOK3) 관광객은 참여 확률이 71% 낮았다.

교육(D_MOK4) 및 기타(D_MOK5) 목적은 여가 목적과 유의한 차이가 없었다.

성별과 연령을 통제했을 때도 이러한 경향은 유지되었으며, 여성은 남성보다 참여 확률이 다소 낮았다.

## 방한 횟수

In [18]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)
var = "D_NUM"            # 방한횟수

# 교차표
ct = pd.crosstab(df[wellness_col], df[var])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 방한횟수 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 방한횟수 ===
D_NUM     1     2     3     4
Q8a04                        
0      6150  2537  1472  3047
1      1456   446   243   446 

Chi2=86.118, dof=3, p-value=1.494e-18


In [19]:
wellness_col = "Q8a04"   # 웰니스 참여 여부
var = "D_NUM"            # 방한횟수
controls = ["D_SEX", "D_AGE"]

y = df[wellness_col]
X = df[[var] + controls].copy()

# 범주형 더미화
X = pd.get_dummies(X, drop_first=True, dtype=float)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.443255
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15793
Method:                           MLE   Df Model:                            3
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                0.006765
Time:                        16:48:34   Log-Likelihood:                -7002.1
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 1.527e-20
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4169      0.072    -19.690      0.000      -1.558      -1.276
D_NUM         -0.1677      0.

로지스틱 회귀분석 결과, 방한횟수가 많아질수록 웰니스 참여 확률은 유의하게 감소하였다(OR=0.85, p<0.001).

반면 여성 관광객은 남성보다 웰니스 참여 확률이 약 18% 더 높았으며(OR=1.18, p<0.001),

연령은 웰니스 참여와 통계적으로 유의한 관련이 없었다.

이는 웰니스 관광이 특히 첫 방문 여성 관광객에게 매력적일 수 있음을 시사한다.

## 한국 방문 횟수

In [20]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)
var = "MVIT"            # 한국 방문 횟수

# 교차표
ct = pd.crosstab(df[wellness_col], df[var])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 방한횟수 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 방한횟수 ===
MVIT     1     2     3    4    5    6    7    8   9    10  ...  16  17  18  \
Q8a04                                                      ...               
0      6150  2537  1472  612  893  197  102  103  20  636  ...   7   4   3   
1      1456   446   243  101  157   29   17   19   5   78  ...   0   0   0   

MVIT   19   20  24  25  27  29   30  
Q8a04                                
0       4  161   2   4   1   1  170  
1       0   12   0   0   0   0   10  

[2 rows x 25 columns] 

Chi2=112.262, dof=24, p-value=2.266e-13


In [21]:
wellness_col = "Q8a04"   # 웰니스 참여 여부
var = "MVIT"            # 방한횟수
controls = ["D_SEX", "D_AGE"]

y = df[wellness_col]
X = df[[var] + controls].copy()

# 범주형 더미화
X = pd.get_dummies(X, drop_first=True, dtype=float)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.443146
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15793
Method:                           MLE   Df Model:                            3
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                0.007008
Time:                        16:48:34   Log-Likelihood:                -7000.4
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 2.803e-21
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5819      0.068    -23.251      0.000      -1.715      -1.449
MVIT          -0.0598      0.

로지스틱 회귀분석 결과, 방한 횟수가 많을수록 웰니스 활동 참여 확률은 유의하게 낮아졌다(OR=0.94, p<0.001).

또한 여성의 참여 확률이 남성보다 높게 나타났으나, 연령은 유의한 영향을 보이지 않았다.

이는 웰니스 관광이 반복 방문자보다는 첫 방문·소수 방문자 중심으로 확산될 가능성이 크다는 점을 보여준다.

## 동반자 유무

In [22]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)
var = "Q7A"            # 동반자 유무

# 교차표
ct = pd.crosstab(df[wellness_col], df[var])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 동반자 유무 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 동반자 유무 ===
Q7A       0     1
Q8a04            
0      3940  9266
1       668  1923 

Chi2=17.029, dof=1, p-value=3.682e-05


In [23]:
wellness_col = "Q8a04"   # 웰니스 참여 여부
var = "Q7A"            # 동반자 유무
controls = ["D_SEX", "D_AGE"]

y = df[wellness_col]
X = df[[var] + controls].copy()

# 범주형 더미화
X = pd.get_dummies(X, drop_first=True, dtype=float)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.445354
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15793
Method:                           MLE   Df Model:                            3
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                0.002060
Time:                        16:48:34   Log-Likelihood:                -7035.3
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 2.186e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7649      0.075    -23.613      0.000      -1.911      -1.618
Q7A            0.1847      0.

로지스틱 회귀분석 결과, 동반자 여부는 웰니스 활동 참여에 유의한 영향을 미쳤다.

동반자와 함께한 관광객은 혼자 여행한 관광객보다 웰니스 참여 확률이 약 20% 더 높았다(OR=1.20, p<0.001).

또한 여성은 남성보다 참여 확률이 14% 더 높았으며(OR=1.14, p=0.003),

연령은 웰니스 참여와 유의한 관련을 보이지 않았다.

## 동반자 유형

In [24]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)

# 방한목적 범주 생성
df["TYP_CAT"] = df[["Q7a_dk", "Q7a2", "Q7a3", "Q7a4", "Q7a5", "Q7a6", "Q7a7", "Q7a8"]].idxmax(axis=1)

# 교차표
ct = pd.crosstab(df[wellness_col], df["TYP_CAT"])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 방한목적 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 방한목적 ===
TYP_CAT  Q7a2  Q7a3  Q7a4  Q7a5  Q7a6  Q7a7  Q7a8  Q7a_dk
Q8a04                                                    
0        2597   794   469   725  3306  1324    51    3940
1         695   169   110   181   575   178    15     668 

Chi2=109.415, dof=7, p-value=1.214e-20


In [25]:
y = df[wellness_col]
# 방한목적
X = df[["Q7a2", "Q7a3", "Q7a4", "Q7a5", "Q7a6", "Q7a7", "Q7a8"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.442471
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15787
Method:                           MLE   Df Model:                            9
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                0.008520
Time:                        16:48:34   Log-Likelihood:                -6989.7
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 1.255e-21
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6192      0.074    -21.763      0.000      -1.765      -1.473
Q7a2           0.4026      0.

동반자 유형은 웰니스 활동 참여에 뚜렷한 차이를 보였다.

혼자 방문자에 비해 배우자/파트너(OR=1.50)와 가족·친지 동반자(OR=1.20)는 웰니스 활동에 더 적극적으로 참여하였으며,

반대로 친구(OR=0.88)와 직장 동료(OR=0.76) 동반자는 참여 확률이 낮았다.

부모님·자녀·기타 동반은 혼자 방문과 차이가 없었다.

또한 여성은 남성보다 참여율이 높았고, 연령이 높을수록 참여율이 다소 낮았다.

## 참여한 활동

In [26]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)

# 참여한 활동 범주 생성 (웰니스 참여 변수 제외)
df["TYP_CAT"] = df[["Q8a01", "Q8a02", "Q8a03", "Q8a05",
                    "Q8a06", "Q8a07", "Q8a08", "Q8a09", "Q8a10",
                    "Q8a11", "Q8a12", "Q8a13", "Q8a14", "Q8a15",
                    "Q8a16", "Q8a17", "Q8a18", "Q8a19", "Q8a20"]].idxmax(axis=1)

# 교차표
ct = pd.crosstab(df[wellness_col], df["TYP_CAT"])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 참여한 활동 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 참여한 활동 ===
TYP_CAT  Q8a01  Q8a02  Q8a03  Q8a05  Q8a06  Q8a07  Q8a08  Q8a09  Q8a10  Q8a11  \
Q8a04                                                                           
0        10035   1772    356    149     32     29     47      6      6     31   
1         1951    482     93     23      8      8      6      2      1      3   

TYP_CAT  Q8a12  Q8a13  Q8a14  Q8a15  Q8a16  Q8a17  Q8a18  Q8a19  Q8a20  
Q8a04                                                                   
0           19      8     23     10     11     81    532     46     13  
1            1      6      1      3      0      0      2      0      1   

Chi2=192.244, dof=18, p-value=3.541e-31


In [27]:
y = df[wellness_col]

# 방한목적
X = df[["Q8a02", "Q8a03", "Q8a05", "Q8a06", "Q8a07",
        "Q8a08", "Q8a09", "Q8a10", "Q8a11", "Q8a12",
        "Q8a13", "Q8a14", "Q8a15", "Q8a16", "Q8a17",
        "Q8a18", "Q8a19", "Q8a20"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.424031
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15776
Method:                           MLE   Df Model:                           20
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                 0.04984
Time:                        16:48:34   Log-Likelihood:                -6698.4
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                5.858e-136
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.1685      0.099    -21.998      0.000      -2.362      -1.975
Q8a02          0.0680      0.

활동 유형별 로지스틱 회귀분석 결과, 웰니스 참여는 전통문화체험(OR=1.35), 자연경관 감상(OR=1.53), 치료·건강검진(OR=1.63), 지역 축제(OR=1.36), 스포츠 관람(OR=1.49) 등과 유의한 정적 관계를 보였다.

반대로 K-POP/드라마 촬영지 방문(OR=0.55), 비즈니스 전문활동(OR=0.41), 국제회의(OR=0.74)는 웰니스 참여 확률을 낮추는 요인으로 나타났다.

성별과 연령은 다른 활동 변수를 통제했을 때는 더 이상 유의하지 않았다.

## 만족한 활동

In [28]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)

# 만족한 활동 범주 생성
df["TYP_CAT"] = df[["Q8_1a1", "Q8_1a2", "Q8_1a3", "Q8_1a4", "Q8_1a5",
                    "Q8_1a6", "Q8_1a7", "Q8_1a8", "Q8_1a9", "Q8_1a10",
                    "Q8_1a11", "Q8_1a12", "Q8_1a13", "Q8_1a14", "Q8_1a15",
                    "Q8_1a16", "Q8_1a17", "Q8_1a18", "Q8_1a19", "Q8_1a20"]].idxmax(axis=1)

# 교차표
ct = pd.crosstab(df[wellness_col], df["TYP_CAT"])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 만족한 활동 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 만족한 활동 ===
TYP_CAT  Q8_1a1  Q8_1a10  Q8_1a11  Q8_1a12  Q8_1a13  Q8_1a14  Q8_1a15  \
Q8a04                                                                   
0          7962       10       41       30       10       32       16   
1          1202        4        5        4        1        4        0   

TYP_CAT  Q8_1a16  Q8_1a17  Q8_1a18  Q8_1a19  Q8_1a2  Q8_1a20  Q8_1a3  Q8_1a4  \
Q8a04                                                                          
0             14       90      579       61    2527       18     981       0   
1              0        1        3        0     679        1     372     177   

TYP_CAT  Q8_1a5  Q8_1a6  Q8_1a7  Q8_1a8  Q8_1a9  
Q8a04                                            
0           563     110      64      80      18  
1           101      18       9      10       0   

Chi2=1302.982, dof=19, p-value=8.096e-265


In [29]:
y = df[wellness_col]

# 만족한 활동
X = df[["Q8_1a1", "Q8_1a2", "Q8_1a3", "Q8_1a5",
        "Q8_1a6", "Q8_1a7", "Q8_1a8", "Q8_1a9", "Q8_1a10",
        "Q8_1a11", "Q8_1a12", "Q8_1a13", "Q8_1a14", "Q8_1a15",
        "Q8_1a16", "Q8_1a17", "Q8_1a18", "Q8_1a19", "Q8_1a20"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.386968
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15775
Method:                           MLE   Df Model:                           21
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                  0.1329
Time:                        16:48:34   Log-Likelihood:                -6112.9
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.3278      0.133     17.446      0.000       2.066       2.589
Q8_1a1        -1.7209      0.

활동 만족도를 독립변수로 한 로지스틱 회귀분석 결과, 대부분의 활동에서 만족도가 높을수록 웰니스 활동 참여 확률은 낮았다.

특히 비즈니스(OR=0.009), 국제회의(OR=0.02), 한류 활동(OR=0.11)은 웰니스 참여와 강한 음의 관계를 보였다.

반면 성별은 유의하지 않았고, 연령이 높을수록 참여 확률이 소폭 낮았다.

이는 웰니스 관광이 다른 활동에 대한 만족도가 낮은 관광객에게 대안적·보완적 성격으로 작용할 수 있음을 시사한다.

## 방문 지역

In [30]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)

# 방문 지역 범주 생성
df["TYP_CAT"] = df[["Q9_2a01", "Q9_2a02", "Q9_2a03", "Q9_2a04", "Q9_2a05",
                    "Q9_2a06", "Q9_2a07", "Q9_2a08", "Q9_2a09", "Q9_2a10",
                    "Q9_2a11", "Q9_2a12", "Q9_2a13", "Q9_2a14", "Q9_2a15",
                    "Q9_2a16", "Q9_2a17"]].idxmax(axis=1)

# 교차표
ct = pd.crosstab(df[wellness_col], df["TYP_CAT"])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 방문 지역 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 방문 지역 ===
TYP_CAT  Q9_2a01  Q9_2a02  Q9_2a03  Q9_2a04  Q9_2a05  Q9_2a06  Q9_2a07  \
Q8a04                                                                    
0          10534      446      254       60       57       47       45   
1           2205       55       47       23        9        3        5   

TYP_CAT  Q9_2a08  Q9_2a09  Q9_2a10  Q9_2a11  Q9_2a12  Q9_2a13  Q9_2a14  \
Q8a04                                                                    
0             15      130      176       84       92      576       27   
1              1        7       20       11       14      131        1   

TYP_CAT  Q9_2a15  Q9_2a16  Q9_2a17  
Q8a04                               
0             27       30      606  
1              8        9       42   

Chi2=108.451, dof=16, p-value=8.829e-16


In [31]:
y = df[wellness_col]

# 방문지역
X = df[["Q9_2a02", "Q9_2a03", "Q9_2a04", "Q9_2a05",
        "Q9_2a06", "Q9_2a07", "Q9_2a08", "Q9_2a09", "Q9_2a10",
        "Q9_2a11", "Q9_2a12", "Q9_2a13", "Q9_2a14", "Q9_2a15",
        "Q9_2a16", "Q9_2a17"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.442950
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15778
Method:                           MLE   Df Model:                           18
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                0.007448
Time:                        16:48:34   Log-Likelihood:                -6997.3
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 2.647e-14
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.7788      0.070    -25.292      0.000      -1.917      -1.641
Q9_2a02        0.0927      0.

방문지역에 따른 웰니스 참여 차이를 살펴본 결과, 인천(OR=1.33), 강원(OR=1.53), 부산(OR=1.42) 방문자는 서울 방문자보다 웰니스 활동 참여 확률이 유의하게 높았다.

반면 경북 방문자는 서울 대비 웰니스 참여 확률이 낮았다(OR=0.62).

성별은 여성의 참여 확률이 높았으나, 연령은 유의한 차이를 보이지 않았다.

## 주요 이용 숙박시설

In [32]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)

# 주요 이용 숙박시설 범주 생성
df["TYP_CAT"] = df[["Q9_5A1", "Q9_5A2", "Q9_5A3", "Q9_5A4", "Q9_5A5",
                    "Q9_5A6", "Q9_5A7", "Q9_5A8", "Q9_5A9"]].idxmax(axis=1)

# 교차표
ct = pd.crosstab(df[wellness_col], df["TYP_CAT"])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 주요 이용 숙박시설 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 주요 이용 숙박시설 ===
TYP_CAT  Q9_5A1  Q9_5A2  Q9_5A3  Q9_5A4  Q9_5A5  Q9_5A6  Q9_5A7  Q9_5A8  \
Q8a04                                                                     
0         10656     323     112     536     180     161     932     286   
1          1840      70      39     133      26      25     387      70   

TYP_CAT  Q9_5A9  
Q8a04            
0            20  
1             1   

Chi2=211.103, dof=8, p-value=2.913e-41


In [33]:
y = df[wellness_col]

# 주요 이용 숙박시설
X = df[["Q9_5A2", "Q9_5A3", "Q9_5A4", "Q9_5A5",
        "Q9_5A6", "Q9_5A7", "Q9_5A8", "Q9_5A9"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.438258
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15786
Method:                           MLE   Df Model:                           10
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                 0.01796
Time:                        16:48:34   Log-Likelihood:                -6923.2
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 1.126e-48
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8004      0.071    -25.344      0.000      -1.940      -1.661
Q9_5A2         0.2806      0.

주요 숙박시설 유형은 웰니스 활동 참여와 유의한 관련이 있었다.

호텔을 기준으로, 콘도/리조트(OR=2.05), 친척·친구 집(OR=2.38), 게스트하우스/호스텔(OR=1.47), 모텔/여관(OR=1.32), 기타 숙박(OR=1.46) 이용자는 웰니스 활동 참여 확률이 더 높았다.

반면 민박·펜션이나 기숙사·연수원은 차이가 없었으며, 숙박하지 않은 경우 참여 확률이 낮은 경향을 보였으나 통계적으로는 유의하지 않았다.

여성은 남성보다 참여 확률이 높았으나, 연령은 유의한 차이를 보이지 않았다.

## 쇼핑항목

In [34]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)

# 쇼핑항목 범주 생성
df["TYP_CAT"] = df[["Q10_2a01", "Q10_2a02", "Q10_2a03", "Q10_2a04", "Q10_2a05",
                    "Q10_2a06", "Q10_2a07", "Q10_2a08", "Q10_2a09", "Q10_2a10",
                    "Q10_2a11", "Q10_2a12", "Q10_2a13"]].idxmax(axis=1)

# 교차표
ct = pd.crosstab(df[wellness_col], df["TYP_CAT"])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 쇼핑항목 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 쇼핑항목 ===
TYP_CAT  Q10_2a01  Q10_2a02  Q10_2a03  Q10_2a04  Q10_2a05  Q10_2a06  Q10_2a07  \
Q8a04                                                                           
0            9340      2078      1000       151        76        67        85   
1            1826       444       222        20        13        11        23   

TYP_CAT  Q10_2a08  Q10_2a09  Q10_2a10  Q10_2a11  Q10_2a12  Q10_2a13  
Q8a04                                                                
0              47        48       118        49        49        98  
1               7         2         8         3         4         8   

Chi2=39.357, dof=12, p-value=9.188e-05


In [35]:
y = df[wellness_col]

# 쇼핑항목
X = df[["Q10_2a02", "Q10_2a03", "Q10_2a04", "Q10_2a05",
        "Q10_2a06", "Q10_2a07", "Q10_2a08", "Q10_2a09", "Q10_2a10",
        "Q10_2a11", "Q10_2a12", "Q10_2a13"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.438130
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15782
Method:                           MLE   Df Model:                           14
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                 0.01825
Time:                        16:48:34   Log-Likelihood:                -6921.1
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 8.938e-47
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9532      0.077    -25.493      0.000      -2.103      -1.803
Q10_2a02       0.0517      0.

쇼핑 항목별 로지스틱 회귀분석 결과, 보석·악세서리(OR=1.55), 전자제품(OR=1.49), 인삼·한약재(OR=1.43), 의류(OR=1.28), 신발(OR=1.37) 등은 웰니스 활동 참여 확률과 유의한 정적 관계를 보였다.

반대로 담배(OR=0.64)와 한류스타 관련상품(OR=0.74)을 구매한 관광객은 웰니스 활동 참여 확률이 낮았다.

이는 웰니스 관광이 건강·자기관리·고급소비 지향적 쇼핑 패턴과 연결되는 반면, 일부 대중문화 소비나 흡연 관련 소비와는 대척적 성격을 가짐을 시사한다.

## 쇼핑장소

In [36]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)

# 쇼핑장소 범주 생성
df["TYP_CAT"] = df[["Q10_3a1", "Q10_3a2", "Q10_3a3", "Q10_3a4", "Q10_3a5",
                    "Q10_3a6", "Q10_3a7", "Q10_3a8"]].idxmax(axis=1)

# 교차표
ct = pd.crosstab(df[wellness_col], df["TYP_CAT"])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 쇼핑장소 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 쇼핑장소 ===
TYP_CAT  Q10_3a1  Q10_3a2  Q10_3a3  Q10_3a4  Q10_3a5  Q10_3a6  Q10_3a7  \
Q8a04                                                                    
0           3618     2450     4379     1124      967      501      142   
1            550      408     1056      324      152       75       23   

TYP_CAT  Q10_3a8  
Q8a04             
0             25  
1              3   

Chi2=127.366, dof=7, p-value=2.23e-24


In [37]:
y = df[wellness_col]

# 쇼핑장소
X = df[["Q10_3a2", "Q10_3a3", "Q10_3a4", "Q10_3a5",
                    "Q10_3a6", "Q10_3a7", "Q10_3a8"]].copy()

# 통제변수: 성별, 연령
X_controls = pd.get_dummies(df[["D_SEX", "D_AGE"]], drop_first=True, dtype=float)

# 합치기
X = pd.concat([X, X_controls], axis=1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀
logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())

# 오즈비(OR) 계산
odds_ratios = pd.DataFrame({
    "OR": np.exp(logit_model.params),
    "p-value": logit_model.pvalues
})
print("\n[오즈비(OR) 표]")
print(odds_ratios)

Optimization terminated successfully.
         Current function value: 0.436535
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Q8a04   No. Observations:                15797
Model:                          Logit   Df Residuals:                    15787
Method:                           MLE   Df Model:                            9
Date:                Tue, 30 Sep 2025   Pseudo R-squ.:                 0.02182
Time:                        16:48:34   Log-Likelihood:                -6895.9
converged:                       True   LL-Null:                       -7049.8
Covariance Type:            nonrobust   LLR p-value:                 6.094e-61
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.2051      0.081    -27.120      0.000      -2.365      -2.046
Q10_3a2       -0.0844      0.

쇼핑 장소별 분석 결과, 대형 쇼핑몰(OR=2.14), 백화점(OR=1.87), 로드샵(OR=1.71), 대형마트(OR=1.38)에서 쇼핑한 관광객은 공항면세점 이용자보다 웰니스 활동 참여 확률이 높았다.

반면 시내면세점은 차이가 없었고, 전통시장은 데이터 특성상 추정이 불안정했다.

성별은 여성이 남성보다 참여 확률이 높았으며, 연령은 증가할수록 참여 확률이 낮아지는 경향이 나타났다.

# 05. 카이제곱검정 (만족도지표)

## 전반적 만족도 (Likert)

In [38]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)
var = "Q11"              # 전반적 만족도

# 교차표
ct = pd.crosstab(df[wellness_col], df[var])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 전반적 만족도 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 전반적 만족도 ===
Q11     1   2    3     4     5
Q8a04                         
0      26  22  378  4878  7902
1       5   1   33   634  1918 

Chi2=190.108, dof=4, p-value=5.024e-40


## 재방문 의사 (Likert)

In [39]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)
var = "Q13"              # 재방문 의사

# 교차표
ct = pd.crosstab(df[wellness_col], df[var])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 재방문 의사 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 재방문 의사 ===
Q13     1    2    3     4     5
Q8a04                          
0      34  157  918  4048  8049
1       4   27  155   620  1785 

Chi2=59.442, dof=4, p-value=3.801e-12


## 타인 추천 의향 (Likert)

In [40]:
wellness_col = "Q8a04"   # 웰니스 참여 여부 (0/1)
var = "Q14"              # 타인 추천 의향

# 교차표
ct = pd.crosstab(df[wellness_col], df[var])

# 카이제곱 검정
chi2, p, dof, expected = stats.chi2_contingency(ct)

print("=== 카이제곱 검정: 웰니스 참여 x 타인 추천 의향 ===")
print(ct, "\n")
print(f"Chi2={chi2:.3f}, dof={dof}, p-value={p:.4g}")

=== 카이제곱 검정: 웰니스 참여 x 타인 추천 의향 ===
Q14    1   2    3     4     5
Q8a04                        
0      7  23  461  4188  8527
1      1   2   54   567  1967 

Chi2=126.034, dof=4, p-value=2.744e-26
